# Titel
## Mauro Schegg
## 3.3.2025

## Einleitung:

## Zielsetzung und Vorgehensweise:
Es soll ein Modell erstellt werden welches Zeitreihenprognosen erstellt. Es soll ein RNN mit LSTM angewendet werden um diese Prognosen zu erstellen. 

## EDA:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch import nn
import torch.optim as optim
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# Einlesen der Daten
dfp = pd.read_csv('Daten/train.csv')
dfwf1 = pd.read_csv('Daten/windforecasts_wf1.csv')
dfwf2 = pd.read_csv('Daten/windforecasts_wf2.csv')
dfwf3 = pd.read_csv('Daten/windforecasts_wf3.csv')
dfwf4 = pd.read_csv('Daten/windforecasts_wf4.csv')
dfwf5 = pd.read_csv('Daten/windforecasts_wf5.csv')
dfwf6 = pd.read_csv('Daten/windforecasts_wf6.csv')
dfwf7 = pd.read_csv('Daten/windforecasts_wf7.csv')

In [ ]:
# Methode zur Umwandlung in datetime für alle DataFrames
def convert_to_datetime(dfs):
	for df in dfs:
		df['date'] = pd.to_datetime(df['date'], format='%Y%m%d%H')
	return dfs

# Liste der DataFrames
dfs = [dfp,dfwf1, dfwf2, dfwf3, dfwf4, dfwf5, dfwf6, dfwf7]

# Aufruf der Funktion
dfs = convert_to_datetime(dfs)

In [ ]:
# Die ersten 5 Zeilen der Leistung der Windparks
dfp.head()

In [ ]:
# Die ersten 5 Zeilen der Windvorhersagen für Windpark 1
dfwf1.head()

**Erkenntnisse:**
- Es werden alle 12h die Winddaten für 48h vorraus gesagt, dies bedeutet, dass es 4 Vorhersagen für die meisten Zeitpunkte gibt.

In [ ]:
# Dimensionen des Datensatzes
print(dfp.shape)
print(dfwf1.shape)

- Das DataFrame für die Leistungen der Windpark hat 8 Spalten eine davon ist die Zeit, das heisst es hat 7 Windparks.

In [ ]:
# Analyse der Datentypen
dfp.dtypes

In [ ]:
# Überprüfen des leistungs DataFrames auf fehlende Werte
print(dfp.isna().sum())

In [ ]:
# Überprüfen des leistungs DataFrames auf fehlende Werte
print(dfwf1.isna().sum())

In [ ]:
# Fehlende Werte durch Mittelwert ersetzen.
# dfs = [dfwf1, dfwf2, dfwf3, dfwf4, dfwf5, dfwf6, dfwf7]
# for i in range(len(dfs)):
    # dfs[i].fillna(dfs[i].mean(), inplace=True)

In [ ]:
dfwf1.isna().sum()

- Es hat keine fehlenden Daten im leistungs Datensatz.
- Es hat $11160$ fehlende Daten bei jeder der Spalten $u,v,w_s,w_d$. Dies sind etwa $11\%$ der gesamten Daten.

### Deskriptive Statistik:

In [ ]:
dfp.describe()
sns.boxplot(dfp)

- Jedes dieser Boxplots beschreibt die normalisierten Daten für die Leistung eines Windparks.
- Windpark 1 scheint Aussreisser zu haben welche betrachtet werden müssen.

### Umwandlung des leistungs Datensatzes:

In [ ]:
# Umwandlung dfp
dfp = dfp.rename(columns={"wp1":"wf1","wp2":"wf2","wp3":"wf3","wp4":"wf4","wp5":"wf5","wp6":"wf6","wp7":"wf7"})
# Zusammenführen der Leistungen in eine Spalte
dfp = dfp.melt(id_vars=["date"], value_name="wpower",var_name="wfarm",ignore_index=True)
dfp["wfarm"] = dfp["wfarm"].astype("string")
dfp.dtypes

In [ ]:
dfp.head()

### Prognosen:

In [ ]:
sns.boxplot(dfwf1)

**Zeitabschnitte**
- hors beschreibt die wie vielte Stunde der Vorhersage das es ist. Dies sind Werte von 1-48.

**Kartesische Koordinaten:**
- u beschreibt die Windgeschwindigkeit in Ost-West-Richtung
- v beschreibt die Windgeschwindigkeit in Nord-Süd-Richtung

**Polar Koordinaten:***
- ws = $\sqrt{u^2+v^2}$ Betrag des Geschwindigkeitsvektors
- wf beschreibt die Richtung des Windes in Grad

In [ ]:
# Liste der DataFrames und zugehörige Keys
dfs = [dfwf1, dfwf2, dfwf3, dfwf4, dfwf5, dfwf6, dfwf7]
keys = ["wf1", "wf2", "wf3", "wf4", "wf5", "wf6", "wf7"]

# Neue DataFrames mit 'wfarm'-Spalte erstellen
dfs = [df.assign(wfarm=key) for df, key in zip(dfs, keys)]

# DataFrames zusammenfügen
dfw = pd.concat(dfs, ignore_index=True)

# Spalten umsortieren, damit 'wfarm' ganz links steht
cols = ["wfarm"] + [col for col in dfw.columns if col != "wfarm"]
dfw = dfw[cols]

# Ergebnis anzeigen
dfw.head()

In [ ]:
dfw["forecast"] = pd.to_datetime(dfw["date"],format="%Y%m%d%H")
dfw["delta_hours"] = pd.to_timedelta(dfw["hors"],unit="hours")
dfw["forecast_date"] = dfw["forecast"]+dfw["delta_hours"]

In [ ]:
# Encoding periodischer Merkmale
# Richtung
dfw["wd_sin"] = np.sin(2*np.pi*dfw["wd"]/360)
dfw["wd_cos"] = np.cos(2**np.pi*dfw["wd"]/360)
# Monat
dfw["month"] = dfw.forecast_date.dt.month
dfw["month_sin"] = np.sin(2**np.pi*dfw["month"]/12)
dfw["month_cos"] = np.cos(2**np.pi*dfw["month"]/12)
# Tag
dfw["day"] = dfw.forecast_date.dt.day
dfw["day_sin"] = np.sin(2**np.pi*dfw["day"]/31)
dfw["day_cos"] = np.cos(2**np.pi*dfw["day"]/31)
# Stunde
dfw["hour"] = dfw.forecast_date.dt.hour
dfw["hour_sin"] = np.sin(2**np.pi*dfw["hour"]/24)
dfw["hour_cos"] = np.cos(2**np.pi*dfw["hour"]/24)


In [ ]:
# Gleitender Mittelwert
hgm = 6 
dfw["u_gm"] = dfw.u.rolling(hgm).mean()
dfw["v_gm"] = dfw.v.rolling(hgm).mean()
dfw["ws_gm"] = dfw.ws.rolling(hgm).mean()

In [ ]:
del dfw["forecast"]

### Merge:

In [ ]:
# Verbindung der beiden Dataframes
df = dfp.merge(dfw,how="outer",left_on=["date","wfarm"],right_on=["forecast_date","wfarm"])  

In [ ]:
del df["date_x"]
del df["date_y"]
del df["hors"]

In [ ]:
df.head(100)

In [ ]:
df.isna().sum()

### Heatmap:

In [ ]:
df.columns

In [ ]:
corr = df[['wpower', 'u', 'v', 'ws', 'wd','wd_sin', 'wd_cos','month_sin', 'month_cos','day_sin',
       'day_cos','hour_sin', 'hour_cos', 'u_gm', 'v_gm', 'ws_gm']].corr()
plt.figure(figsize=(12,10))
sns.heatmap(corr,annot=True,cmap="viridis")

In [ ]:
# sns.pairplot(df.sample(n=250))

## Random Tree um die wichtigsten Features zu finden:

In [ ]:
dfTest = df.sample(2000)
dfTest.dropna(axis=0,inplace=True)
dfTest.isna().sum()

In [ ]:
dfTest.head()

In [ ]:
X = dfTest[['u', 'v', 'ws', 'wd','wd_sin', 'wd_cos','month_sin', 'month_cos',
       'day_cos','hour_sin', 'hour_cos', 'u_gm']]
y = dfTest["wpower"]

# Entscheidungsbaum-Regression mit max_depth
forest = RandomForestRegressor(n_estimators=1000,max_depth=250,random_state=None)
forest.fit(X, y)

In [ ]:
feature_importance = forest.feature_importances_
importance_df = pd.DataFrame({"Features":X.columns,"Importance":feature_importance})
importance_df.sort_values(by="Importance",ascending=False)

In [ ]:
top_features = importance_df.sort_values(by="Importance",ascending=False).head(6)
top_features

In [ ]:
plt.barh(top_features["Features"],top_features["Importance"])
plt.xlabel("Feature Importance")
plt.ylabel("Feature Name")
plt.title("Feature Importance der einzelnen Komponenten")
plt.gca().invert_yaxis()
plt.grid()

- Wie zu erwarten ist die Windgeschwindigkeit einer der wichtigsten Faktoren für die Leistung eines Windkraftwerks.
- $ws_{gm}$, $v_{gm}$, $day_{sin}$ wurden entfernt das sie sonst doppelt representiert sind.

## Bewertungskriterium:
- Es wird danach bewertet wie gut das Modell die Leistung vorraus sagt.
- Dazu wird die **Root Mean Squared Error (RMSE)** Loss Funktion verwendet, da sie grosse Fehler bestraft und das Ergebnis in der Einheit der Leistung interpretiert werden kann.

## Bewertungsmethode
- Holdout-Methode kann verwendet werden, da der Datensatz sehr gross ist.

## Baseline-Modell:
- Für das Baseline-Modell wird ein LTSM verwendet welches speziell für Zeitabhängige Daten entwickelt wurde. RNN haben das *Vanishing Gradient* Problem, dies hat LTSM nicht. 
- Das *Vanishing Gradient* Problem ist wenn die Gradienten während des Backpropagation-Trainings immer kleiner werden.

In [ ]:

# Parameter für die LSTM-Eingabe
TIME_STEPS = 48  # Anzahl vergangener Stunden für die Vorhersage der nächsten Stunde

# Modell-Parameter
input_size = 6  # Anzahl der Features
output_size = 1
learning_rates = [0.1, 0.01, 0.001, 0.0001, 0.00001]
hidden_sizes = [50, 100, 150]
num_layers_list = [2, 3, 4]  # Verschiedene Anzahl der LSTM-Schichten

# LSTM Modell definieren
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        return self.fc(lstm_out[:, -1, :])  # Nur letztes Zeitfenster nutzen

# Modell, Verlustfunktion und RMSE-Definition
criterion = nn.MSELoss()

def compute_rmse(y_pred, y_true):
    mse = criterion(y_pred, y_true)
    rmse = torch.sqrt(mse)  # RMSE Loss Funktion
    return rmse

# Liste der Resultate 
results = []

# Iteration über alle Windparks und Kombinationen der Hyperparameter
for wf_id in dfTest["wfarm"].unique():
    print(f"Training für Windpark {wf_id}...")

    # Filtern der Daten für den Windpark
    df_wf = dfTest[dfTest["wfarm"] == "wf1"]

    # Features und Zielvariable
    X = torch.tensor(df_wf[["ws", "u_gm", "v", "day_cos", "hour_cos", "month_sin"]].values, dtype=torch.float32)
    y = torch.tensor(df_wf["wpower"].values, dtype=torch.float32).view(-1, 1)  # 2D-Shape für LSTM

    # Min-Max-Normalisierung mit PyTorch (statt Sklearn)
    y_min, y_max = y.min(), y.max()
    y_scaled = (y - y_min) / (y_max - y_min)

    # Funktion zur Erstellung von Sequenzen für das LSTM
    def create_sequences(X, y, time_steps):
        Xs, ys = [], []
        for i in range(len(X) - time_steps):
            Xs.append(X[i:i + time_steps])  # 48 Werte als Eingabe
            ys.append(y[i + time_steps])    # Der nächste Wert als Ziel
        return torch.stack(Xs), torch.stack(ys)

    # Erstelle die Sequenzen
    X_seq, y_seq = create_sequences(X, y_scaled, TIME_STEPS)

    # Holdout-Methode: Train (70%), Validation (10%), Test (20%) Split
    train_split = int(0.7 * len(X_seq))
    val_split = int(0.8 * len(X_seq))  # 70% + 10% = 80%

    X_train, X_val, X_test = X_seq[:train_split], X_seq[train_split:val_split], X_seq[val_split:]
    y_train, y_val, y_test = y_seq[:train_split], y_seq[train_split:val_split], y_seq[val_split:]

    # Prüfe die Shapes
    print(f"X_train Shape: {X_train.shape}, y_train Shape: {y_train.shape}")
    print(f"X_val Shape: {X_val.shape}, y_val Shape: {y_val.shape}")
    print(f"X_test Shape: {X_test.shape}, y_test Shape: {y_test.shape}")

    # Testen der verschiedenen Lernraten, versteckten Schichtgrößen und LSTM-Schichten
    for lr in learning_rates:
        for hidden_size in hidden_sizes:
            for num_layers in num_layers_list:
                print(f"Training mit Lernrate: {lr}, Hidden Size: {hidden_size}, num_layers: {num_layers}...")

                # Modell neu erstellen mit den aktuellen Hyperparametern
                model = LSTMModel(input_size, hidden_size, num_layers, output_size)
                optimizer = optim.Adam(model.parameters(), lr=lr)

                # Training
                EPOCHS = 50
                for epoch in range(EPOCHS):
                    model.train()
                    optimizer.zero_grad()
                    y_pred = model(X_train)
                    loss = compute_rmse(y_pred, y_train)
                    loss.backward()
                    optimizer.step()

                    # Validierung
                    model.eval()
                    with torch.no_grad():
                        val_pred = model(X_val)
                        val_loss = compute_rmse(val_pred, y_val)

                    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")

                # Modell testen
                model.eval()
                with torch.no_grad():
                    y_test_pred = model(X_test)

                # Test-Fehler berechnen
                test_loss = compute_rmse(y_test_pred, y_test)
                print(f"Test Loss für Windpark {wf_id} mit Lernrate {lr}, Hidden Size {hidden_size}, und num_layers {num_layers}: {test_loss.item():.4f}")

                # Ergebnisse speichern
                results.append({
                    "wf_id": wf_id,
                    "learning_rate": lr,
                    "hidden_size": hidden_size,
                    "num_layers": num_layers,
                    "test_loss": test_loss.item()
                })

# Speichern der Ergebnisse in einem DataFrame
results_df = pd.DataFrame(results)

# Ausgabe der Resultate
print(results_df)


In [ ]:
print(results_df.columns)

In [ ]:
results_df_sort = results_df.sort_values(by='test_loss',ascending=True)
results_df_sort

In [ ]:
"""
import torch
import torch.nn as nn
import torch.optim as optim

# Parameter für die LSTM-Eingabe
TIME_STEPS = 48  # Anzahl vergangener Stunden für die Vorhersage der nächsten Stunde

# Modell-Parameter
input_size = 6  # Anzahl der Features
hidden_size = 80
num_layers = 3
output_size = 1
learning_rates = [0.1,0.01,0.001,0.0001,0.00001]
hidden_sizes = [50, 100, 150]

# LSTM Modell definieren
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        return self.fc(lstm_out[:, -1, :])  # Nur letztes Zeitfenster nutzen
    
# Modell, Verlustfunktion & 
model = LSTMModel(input_size, hidden_size, num_layers, output_size)
criterion = nn.MSELoss()

def compute_rmse(y_pred, y_true):
    mse = criterion(y_pred, y_true)
    rmse = torch.sqrt(mse) # RMSE Loss Funktion
    return rmse

optimizer = optim.Adam(model.parameters(), lr=0.001)

# Iteration über alle Windparks
for wf_id in dfTest["wfarm"].unique():
    print(f"Training für Windpark {wf_id}...")
    
    # Filtern der Daten für den Windpark
    df_wf = dfTest[dfTest["wfarm"] == "wf1"]
    
    # Features und Zielvariable
    X = torch.tensor(df_wf[["ws", "u_gm", "v", "day_cos", "hour_cos", "month_sin"]].values, dtype=torch.float32)
    y = torch.tensor(df_wf["wpower"].values, dtype=torch.float32).view(-1, 1)  # 2D-Shape für LSTM

    # Min-Max-Normalisierung mit PyTorch (statt Sklearn)
    y_min, y_max = y.min(), y.max()
    y_scaled = (y - y_min) / (y_max - y_min)

    # Funktion zur Erstellung von Sequenzen für das LSTM
    def create_sequences(X, y, time_steps):
        Xs, ys = [], []
        for i in range(len(X) - time_steps):
            Xs.append(X[i:i + time_steps])  # 48 Werte als Eingabe
            ys.append(y[i + time_steps])    # Der nächste Wert als Ziel
        return torch.stack(Xs), torch.stack(ys)

    # Erstelle die Sequenzen
    X_seq, y_seq = create_sequences(X, y_scaled, TIME_STEPS)

				# Holdout-MEthode
    # Train (70%), Validation (10%), Test (20%) Split in PyTorch
    train_split = int(0.7 * len(X_seq))
    val_split = int(0.8 * len(X_seq))  # 70% + 10% = 80%

    X_train, X_val, X_test = X_seq[:train_split], X_seq[train_split:val_split], X_seq[val_split:]
    y_train, y_val, y_test = y_seq[:train_split], y_seq[train_split:val_split], y_seq[val_split:]

    # Prüfe die Shapes
    print(f"X_train Shape: {X_train.shape}, y_train Shape: {y_train.shape}")
    print(f"X_val Shape: {X_val.shape}, y_val Shape: {y_val.shape}")
    print(f"X_test Shape: {X_test.shape}, y_test Shape: {y_test.shape}")

    # Training
    EPOCHS = 50
    for epoch in range(EPOCHS):
        model.train()
        optimizer.zero_grad()
        y_pred = model(X_train)
        loss = compute_rmse(y_pred, y_train)
        loss.backward()
        optimizer.step()
        
        # Validierung
        model.eval()
        with torch.no_grad():
            val_pred = model(X_val)
            val_loss = compute_rmse(val_pred, y_val)
        
        print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")

    # Modell testen
    model.eval()
    with torch.no_grad():
        y_test_pred = model(X_test)

    # Test-Fehler berechnen sollte niedrig sein
    test_loss = compute_rmse(y_test_pred, y_test)
    print(f"Test Loss für Windpark {wf_id}: {test_loss.item():.4f}")
"""